In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import yaml
from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab
import json

def read_yaml(path):
    with open(path, 'rb') as yaml_file:
        return yaml.safe_load(yaml_file)


In [2]:
df = pd.read_csv("../dataset/UCI_Credit_Card.csv")
df.rename(columns={'PAY_0': 'PAY_1', 'SEX': 'GENDER', 'default.payment.next.month': 'DEFAULTS'}, inplace=True)
df.drop(columns=['ID'], axis=1, inplace=True)
df.head().T

,0,1,2,3,4
LIMIT_BAL,20000.0,120000.0,90000.0,50000.0,50000.0
GENDER,2.0,2.0,2.0,2.0,1.0
EDUCATION,2.0,2.0,2.0,2.0,2.0
MARRIAGE,1.0,2.0,2.0,1.0,1.0
AGE,24.0,26.0,34.0,37.0,57.0
PAY_1,2.0,-1.0,0.0,0.0,-1.0
PAY_2,2.0,2.0,0.0,0.0,0.0
PAY_3,-1.0,0.0,0.0,0.0,-1.0
PAY_4,-1.0,0.0,0.0,0.0,0.0
PAY_5,-2.0,0.0,0.0,0.0,0.0


In [25]:
df['GENDER'].replace({'Male':1, 'Female':2}, inplace=True)
df['GENDER']

0        2
1        2
2        2
3        2
4        1
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: GENDER, Length: 30000, dtype: int64

In [5]:
config_data = read_yaml("../config/schema.yaml")
cat_col = config_data.get("domain_value")
# if not cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and not cat_col.get("EDUCATION") in df.EDUCATION.unique():
#         df["MARRIAGE"].replace(to_replace=0, value=3, inplace=True)
#         df["EDUCATION"].replace(to_replace=[0, 5, 6], value=4, inplace=True)

In [6]:
cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and cat_col.get("EDUCATION") in df.EDUCATION.unique()

C:\Users\HP\AppData\Local\Temp\ipykernel_20856\2620371174.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and cat_col.get("EDUCATION") in df.EDUCATION.unique()


False

In [7]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["DEFAULTS"], axis=1)
y = df["DEFAULTS"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25)

In [8]:
for df in (train_x, test_x):
    if not cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and not cat_col.get("EDUCATION") in df.EDUCATION.unique():
            df["MARRIAGE"].replace(to_replace=0, value=3, inplace=True)
            df["EDUCATION"].replace(to_replace=[0, 5, 6], value=4, inplace=True)
            print(True)
    else:
        print(False)

True
True


C:\Users\HP\AppData\Local\Temp\ipykernel_20856\283351617.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if not cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and not cat_col.get("EDUCATION") in df.EDUCATION.unique():


In [9]:
train_x.EDUCATION.value_counts()

2    10505
1     7927
3     3702
4      366
Name: EDUCATION, dtype: int64

In [10]:
profile = Profile(sections=[DataDriftProfileSection()])

In [12]:
train_df = pd.read_csv("../CreditCard_Defaults/artifact/data_ingestion/2022-09-15-16-54-11/ingested_data/train/UCI_Credit_Card.csv")
test_df = pd.read_csv("../CreditCard_Defaults/artifact/data_ingestion/2022-09-15-16-54-11/ingested_data/test/UCI_Credit_Card.csv")

In [13]:
profile.calculate(train_df, test_df)

c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:43: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  n_values = reference_data.append(current_data).nunique()
c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:54: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  n_values = reference_data.append(current_data).nunique()
c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:43: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  n_values = reference_data.append(current_data).nunique()
c:\Users\HP\Desktop\Credit-C

In [14]:
report = json.loads(profile.json())

In [15]:
report_file_path = "../Notebook/D_Drift.json"
with open(report_file_path, "w") as report_file:
    json.dump(report, report_file, indent=6)

In [17]:
dashboard = Dashboard(tabs=[DataDriftTab()])
dashboard.calculate(train_df, test_df)
dashboard.save("D_Drift.html")

c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:43: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:54: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:43: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\HP\Desktop\Credit-Card-Default-Prediction-ML-Project\venv\lib\site-packages\evidently\analyzers\stattests\registry.py:54: FutureWarning:

The series.append method is deprecated and will be r